In [1]:
import torch
import tqdm
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

import networkx as nx
import matplotlib.pyplot as plt
import scipy.sparse
import scipy.sparse.linalg ## for some reason this didn't import for eigsh without this line
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from numpy.linalg import solve

import numpy as np
import sys
from MARBLE import plotting, preprocessing, dynamics, net, postprocessing
import matplotlib.pyplot as plt

In [2]:
root_dir = os.path.expanduser('~')
dat_dir = os.path.join(root_dir,'projects/BCI_Kaggle/dataset_v2_blocks/dataset_v2_blocks')

In [3]:
import os
import numpy as np
import pandas as pd

def extract_metadata(data_dict, base_key=[], metadata_rows=None):
    if metadata_rows is None:
        metadata_rows = []

    for key, value in data_dict.items():
        current_key = base_key + [key]
        if isinstance(value, dict):
            extract_metadata(value, current_key, metadata_rows)
        else:
            # If value is not a dict, we are at the file level
            row = dict(zip(range(len(current_key)), current_key))
            row[len(current_key)] = value
            metadata_rows.append(row)

    return metadata_rows

def load_npz_files(base_directory):
    data_dict = {}

    for root, _, files in os.walk(base_directory):
        for file in files:
            if file.endswith('.npz'):
                file_path = os.path.join(root, file)
                
                # Extract metadata from file path
                path_parts = file_path.split(os.sep)
                # Get the relevant part of the path
                relevant_parts = path_parts[-8:]
                
                # Load npz file
                data = np.load(file_path)
                data_dict_entry = {key: data[key] for key in data.files}
                
                # Construct the hierarchical dictionary
                current_dict = data_dict
                for part in relevant_parts[:-1]:
                    if part not in current_dict:
                        current_dict[part] = {}
                    current_dict = current_dict[part]
                
                current_dict[relevant_parts[-1]] = data_dict_entry

    return data_dict

base_directory = "/msc/home/vsharm64/projects/BCI_Kaggle/dataset_v2_blocks/dataset_v2_blocks"
data_dict = load_npz_files(base_directory)

# Extract metadata
metadata_rows = extract_metadata(data_dict)

# Create a DataFrame from the list of rows
metadata_df = pd.DataFrame(metadata_rows).sort_index(axis=1)

# Rename columns to more meaningful names
metadata_df.columns = [f'level_{i}' for i in metadata_df.columns]

# Display the DataFrame
metadata_df.head()


,level_0,level_1,level_2,level_3,level_4,level_5,level_6,level_7,level_8,level_9
0,dataset_v2_blocks,dataset_v2_blocks,health,left,alex_kovalev_standart_elbow_left,preproc_angles,train,0037.npz,data_myo,"[[-0.12941176470588234, 0.1450980392156862, 0...."
1,dataset_v2_blocks,dataset_v2_blocks,health,left,alex_kovalev_standart_elbow_left,preproc_angles,train,0037.npz,data_vr,"[[[0.0, 0.0, 0.0, -1.0], [-0.03047309966664615..."
2,dataset_v2_blocks,dataset_v2_blocks,health,left,alex_kovalev_standart_elbow_left,preproc_angles,train,0037.npz,data_angles,"[[0.30133037585990835, -0.14699556169912562, 0..."
3,dataset_v2_blocks,dataset_v2_blocks,health,left,alex_kovalev_standart_elbow_left,preproc_angles,train,0037.npz,myo_ts,"[1331098.9952673502, 1331099.041548939, 133109..."
4,dataset_v2_blocks,dataset_v2_blocks,health,left,alex_kovalev_standart_elbow_left,preproc_angles,train,0121.npz,data_myo,"[[-0.11372549019607847, -0.027450980392156876,..."


In [4]:
metadata_df.iloc[:,:8].describe()

,level_0,level_1,level_2,level_3,level_4,level_5,level_6,level_7
count,12404,12404,12404,12404,12404,12404,12404,12404
unique,1,1,2,2,22,1,4,144
top,dataset_v2_blocks,dataset_v2_blocks,health,left,matthew_antonov_standart_elbow_left,preproc_angles,train,0006.npz
freq,12404,12404,11128,6716,864,12404,8592,138


In [5]:
metadata_df['level_6'].unique()

array(['train', 'test', 'submit', 'train_strong_activity'], dtype=object)

In [6]:
metadata_df.loc[metadata_df['level_6']=='submit']

,level_0,level_1,level_2,level_3,level_4,level_5,level_6,level_7,level_8,level_9
11704,dataset_v2_blocks,dataset_v2_blocks,amputant,left,fedya_tropin_standart_elbow_left,preproc_angles,submit,0037.npz,data_myo,"[[0.19215686274509802, 0.0039215686274509665, ..."
11705,dataset_v2_blocks,dataset_v2_blocks,amputant,left,fedya_tropin_standart_elbow_left,preproc_angles,submit,0037.npz,myo_ts,"[337481.88911415, 337481.93570765737, 337481.9..."
11706,dataset_v2_blocks,dataset_v2_blocks,amputant,left,fedya_tropin_standart_elbow_left,preproc_angles,submit,0030.npz,data_myo,"[[-0.06666666666666665, -0.05882352941176472, ..."
11707,dataset_v2_blocks,dataset_v2_blocks,amputant,left,fedya_tropin_standart_elbow_left,preproc_angles,submit,0030.npz,myo_ts,"[337239.4001226501, 337239.4464885098, 337239...."
11708,dataset_v2_blocks,dataset_v2_blocks,amputant,left,fedya_tropin_standart_elbow_left,preproc_angles,submit,0053.npz,data_myo,"[[-0.21568627450980393, -0.0039215686274509665..."
...,...,...,...,...,...,...,...,...,...,...
11843,dataset_v2_blocks,dataset_v2_blocks,amputant,left,fedya_tropin_standart_elbow_left,preproc_angles,submit,0020.npz,myo_ts,"[336939.7717833501, 336939.8205107955, 336939...."
11844,dataset_v2_blocks,dataset_v2_blocks,amputant,left,fedya_tropin_standart_elbow_left,preproc_angles,submit,0039.npz,data_myo,"[[-0.0039215686274509665, 0.003921568627450966..."
11845,dataset_v2_blocks,dataset_v2_blocks,amputant,left,fedya_tropin_standart_elbow_left,preproc_angles,submit,0039.npz,myo_ts,"[337558.88148415, 337558.929068199, 337558.944..."
11846,dataset_v2_blocks,dataset_v2_blocks,amputant,left,fedya_tropin_standart_elbow_left,preproc_angles,submit,0055.npz,data_myo,"[[0.0039215686274509665, -0.003921568627450966..."


In [7]:
metadata_df#.loc[metadata_df['level_6']=='train']

,level_0,level_1,level_2,level_3,level_4,level_5,level_6,level_7,level_8,level_9
0,dataset_v2_blocks,dataset_v2_blocks,health,left,alex_kovalev_standart_elbow_left,preproc_angles,train,0037.npz,data_myo,"[[-0.12941176470588234, 0.1450980392156862, 0...."
1,dataset_v2_blocks,dataset_v2_blocks,health,left,alex_kovalev_standart_elbow_left,preproc_angles,train,0037.npz,data_vr,"[[[0.0, 0.0, 0.0, -1.0], [-0.03047309966664615..."
2,dataset_v2_blocks,dataset_v2_blocks,health,left,alex_kovalev_standart_elbow_left,preproc_angles,train,0037.npz,data_angles,"[[0.30133037585990835, -0.14699556169912562, 0..."
3,dataset_v2_blocks,dataset_v2_blocks,health,left,alex_kovalev_standart_elbow_left,preproc_angles,train,0037.npz,myo_ts,"[1331098.9952673502, 1331099.041548939, 133109..."
4,dataset_v2_blocks,dataset_v2_blocks,health,left,alex_kovalev_standart_elbow_left,preproc_angles,train,0121.npz,data_myo,"[[-0.11372549019607847, -0.027450980392156876,..."
...,...,...,...,...,...,...,...,...,...,...
12399,dataset_v2_blocks,dataset_v2_blocks,amputant,left,valery_first_standart_elbow_left,preproc_angles,train_strong_activity,0039.npz,myo_ts,"[1339066.40039925, 1339066.4488972398, 1339066..."
12400,dataset_v2_blocks,dataset_v2_blocks,amputant,left,valery_first_standart_elbow_left,preproc_angles,train_strong_activity,0055.npz,data_myo,"[[0.1843137254901961, -0.03529411764705881, 0...."
12401,dataset_v2_blocks,dataset_v2_blocks,amputant,left,valery_first_standart_elbow_left,preproc_angles,train_strong_activity,0055.npz,data_vr,"[[[0.0, 0.0, 0.0, -1.0], [-0.05675145483850774..."
12402,dataset_v2_blocks,dataset_v2_blocks,amputant,left,valery_first_standart_elbow_left,preproc_angles,train_strong_activity,0055.npz,data_angles,"[[-0.7271111805441305, -0.012663524826067493, ..."


In [8]:
metadata_df.iloc[0]['level_9'].shape

(3722, 8)

In [9]:
to_onehot_cols = ['level_2','level_3','level_4']#,'level_7']

In [10]:
# Perform one-hot encoding
onehot_encoded_df = pd.get_dummies(metadata_df[to_onehot_cols])
onehot_encoded_df

,level_2_amputant,level_2_health,level_3_left,level_3_right,level_4_alex_kovalev_standart_elbow_left,level_4_alex_kovalev_standart_elbow_right,level_4_andrew_snailbox_standart_elbow_right,level_4_anna_makarova_standart_elbow_left,level_4_anna_makarova_standart_elbow_right,level_4_artem_snailbox_standart_elbow_left,...,level_4_misha_korobok_standart_elbow_right,level_4_nikita_snailbox_standart_elbow_left,level_4_nikita_snailbox_standart_elbow_right,level_4_petya_chizhov_standart_elbow_left,level_4_petya_chizhov_standart_elbow_right,level_4_polina_maksimova_standart_elbow_left,level_4_polina_maksimova_standart_elbow_right,level_4_sema_duplin_standart_elbow_left,level_4_sema_duplin_standart_elbow_right,level_4_valery_first_standart_elbow_left
0,False,True,True,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,True,True,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,True,True,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,True,True,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,True,True,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12399,True,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
12400,True,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
12401,True,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
12402,True,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [11]:
metadata_df['OneHot'] = onehot_encoded_df.apply(lambda row: [np.array(row,dtype=float)],axis=1)
metadata_df

,level_0,level_1,level_2,level_3,level_4,level_5,level_6,level_7,level_8,level_9,OneHot
0,dataset_v2_blocks,dataset_v2_blocks,health,left,alex_kovalev_standart_elbow_left,preproc_angles,train,0037.npz,data_myo,"[[-0.12941176470588234, 0.1450980392156862, 0....","[[0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,..."
1,dataset_v2_blocks,dataset_v2_blocks,health,left,alex_kovalev_standart_elbow_left,preproc_angles,train,0037.npz,data_vr,"[[[0.0, 0.0, 0.0, -1.0], [-0.03047309966664615...","[[0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,..."
2,dataset_v2_blocks,dataset_v2_blocks,health,left,alex_kovalev_standart_elbow_left,preproc_angles,train,0037.npz,data_angles,"[[0.30133037585990835, -0.14699556169912562, 0...","[[0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,..."
3,dataset_v2_blocks,dataset_v2_blocks,health,left,alex_kovalev_standart_elbow_left,preproc_angles,train,0037.npz,myo_ts,"[1331098.9952673502, 1331099.041548939, 133109...","[[0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,..."
4,dataset_v2_blocks,dataset_v2_blocks,health,left,alex_kovalev_standart_elbow_left,preproc_angles,train,0121.npz,data_myo,"[[-0.11372549019607847, -0.027450980392156876,...","[[0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,..."
...,...,...,...,...,...,...,...,...,...,...,...
12399,dataset_v2_blocks,dataset_v2_blocks,amputant,left,valery_first_standart_elbow_left,preproc_angles,train_strong_activity,0039.npz,myo_ts,"[1339066.40039925, 1339066.4488972398, 1339066...","[[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
12400,dataset_v2_blocks,dataset_v2_blocks,amputant,left,valery_first_standart_elbow_left,preproc_angles,train_strong_activity,0055.npz,data_myo,"[[0.1843137254901961, -0.03529411764705881, 0....","[[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
12401,dataset_v2_blocks,dataset_v2_blocks,amputant,left,valery_first_standart_elbow_left,preproc_angles,train_strong_activity,0055.npz,data_vr,"[[[0.0, 0.0, 0.0, -1.0], [-0.05675145483850774...","[[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
12402,dataset_v2_blocks,dataset_v2_blocks,amputant,left,valery_first_standart_elbow_left,preproc_angles,train_strong_activity,0055.npz,data_angles,"[[-0.7271111805441305, -0.012663524826067493, ...","[[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [12]:
train_subset = metadata_df.loc[metadata_df['level_6']=='train']
test_subset = metadata_df.loc[metadata_df['level_6']=='test']
submit_subset = metadata_df.loc[metadata_df['level_6']=='submit']

train_inputs = train_subset.loc[train_subset['level_8']=='data_myo']
train_outputs = train_subset.loc[train_subset['level_8']=='data_angles']

test_inputs = test_subset.loc[test_subset['level_8']=='data_myo']
test_outputs = test_subset.loc[test_subset['level_8']=='data_angles']

submit_inputs = submit_subset.loc[submit_subset['level_8']=='data_myo']

In [13]:
train_inputs

,level_0,level_1,level_2,level_3,level_4,level_5,level_6,level_7,level_8,level_9,OneHot
0,dataset_v2_blocks,dataset_v2_blocks,health,left,alex_kovalev_standart_elbow_left,preproc_angles,train,0037.npz,data_myo,"[[-0.12941176470588234, 0.1450980392156862, 0....","[[0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,..."
4,dataset_v2_blocks,dataset_v2_blocks,health,left,alex_kovalev_standart_elbow_left,preproc_angles,train,0121.npz,data_myo,"[[-0.11372549019607847, -0.027450980392156876,...","[[0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,..."
8,dataset_v2_blocks,dataset_v2_blocks,health,left,alex_kovalev_standart_elbow_left,preproc_angles,train,0030.npz,data_myo,"[[-0.2313725490196078, -0.05882352941176472, -...","[[0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,..."
12,dataset_v2_blocks,dataset_v2_blocks,health,left,alex_kovalev_standart_elbow_left,preproc_angles,train,0053.npz,data_myo,"[[0.10588235294117654, 0.027450980392156765, -...","[[0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,..."
16,dataset_v2_blocks,dataset_v2_blocks,health,left,alex_kovalev_standart_elbow_left,preproc_angles,train,0025.npz,data_myo,"[[0.05882352941176472, 0.03529411764705892, 0....","[[0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,..."
...,...,...,...,...,...,...,...,...,...,...,...
12108,dataset_v2_blocks,dataset_v2_blocks,amputant,left,valery_first_standart_elbow_left,preproc_angles,train,0033.npz,data_myo,"[[0.05882352941176472, 0.027450980392156765, -...","[[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
12112,dataset_v2_blocks,dataset_v2_blocks,amputant,left,valery_first_standart_elbow_left,preproc_angles,train,0068.npz,data_myo,"[[-0.13725490196078427, -0.1686274509803921, -...","[[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
12116,dataset_v2_blocks,dataset_v2_blocks,amputant,left,valery_first_standart_elbow_left,preproc_angles,train,0020.npz,data_myo,"[[0.0039215686274509665, 0.08235294117647052, ...","[[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
12120,dataset_v2_blocks,dataset_v2_blocks,amputant,left,valery_first_standart_elbow_left,preproc_angles,train,0039.npz,data_myo,"[[0.09019607843137245, 0.050980392156862786, 0...","[[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [14]:
train_inputs.iloc[0]['level_9'].shape

(3722, 8)

In [15]:
train_inputs.iloc[0]['OneHot'][0].shape

(26,)

In [16]:
def expand_and_concatenate(row):
    level_9 = row['level_9']
    onehot = row['OneHot']
    
    # Repeat the onehot array to match the number of samples in level_9
    expanded_onehot = np.tile(onehot, (level_9.shape[0], 1))
    
    # Concatenate the level_9 and expanded onehot arrays along the last dimension
    concatenated_array = np.concatenate((level_9, expanded_onehot), axis=1)
    
    return concatenated_array

# Apply the function to each row and store the result in a new column
train_inputs['combined'] = pd.Series(train_inputs.apply(expand_and_concatenate, axis=1))
test_inputs['combined'] = pd.Series(test_inputs.apply(expand_and_concatenate, axis=1))
submit_inputs['combined'] = pd.Series(submit_inputs.apply(expand_and_concatenate, axis=1))

# Verify the shape of the combined array
print(train_inputs.iloc[0]['combined'].shape)  # Should print (3722, 178)
print(test_inputs.iloc[0]['combined'].shape)
print(submit_inputs.iloc[0]['combined'].shape)  # Should print (3722, 178)

# Print the combined array for verification
print(train_inputs['combined'].iloc[0])

/tmp/ipykernel_2894668/1065248138.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_inputs['combined'] = pd.Series(train_inputs.apply(expand_and_concatenate, axis=1))


(3722, 34)
(3729, 34)
(3725, 34)
[[-0.12941176  0.14509804  0.06666667 ...  0.          0.
   0.        ]
 [ 0.17647059  0.00392157 -0.01176471 ...  0.          0.
   0.        ]
 [-0.16862745 -0.05098039 -0.0745098  ...  0.          0.
   0.        ]
 ...
 [ 0.00392157 -0.01176471 -0.02745098 ...  0.          0.
   0.        ]
 [-0.15294118 -0.01960784 -0.01176471 ...  0.          0.
   0.        ]
 [ 0.08235294 -0.05098039 -0.00392157 ...  0.          0.
   0.        ]]


/tmp/ipykernel_2894668/1065248138.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_inputs['combined'] = pd.Series(test_inputs.apply(expand_and_concatenate, axis=1))
/tmp/ipykernel_2894668/1065248138.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submit_inputs['combined'] = pd.Series(submit_inputs.apply(expand_and_concatenate, axis=1))


# NTK

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from numpy.linalg import solve
import numpy as np
import tqdm

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class NTK:
    def __init__(self, reg=1, sigma=1.0, alpha=0.5):
        self.reg = reg
        self.sigma = sigma
        self.alpha = alpha
        self.sol = None
        self.Xtrain = None
        self.ytrain = None

    def ntk_kernel(self, pair1, pair2):
        out = pair1 @ pair2.T + 1
        N1 = np.sum(np.power(pair1, 2), axis=-1).reshape(-1, 1) + 1
        N2 = np.sum(np.power(pair2, 2), axis=-1).reshape(-1, 1) + 1

        XX = np.sqrt(N1 @ N2.T)
        out = out / XX

        out = np.clip(out, -1, 1)

        first = (
            1
            / np.pi
            * (out * (np.pi - np.arccos(out)) + np.sqrt(1.0 - np.power(out, 2)))
            * XX
        )
        sec = 1 / np.pi * out * (np.pi - np.arccos(out)) * XX
        out = first + sec

        C = 1
        return out / C

    def gaussian_kernel(self, pair1, pair2):
        sq_dist = np.sum(pair1**2, axis=1).reshape(-1, 1) + np.sum(pair2**2, axis=1) - 2 * np.dot(pair1, pair2.T)
        return np.exp(-sq_dist / (2 * self.sigma**2))

    def kernel(self, pair1, pair2):
        ntk = self.ntk_kernel(pair1, pair2)
        gaussian = self.gaussian_kernel(pair1, pair2)
        return self.alpha * ntk + (1 - self.alpha) * gaussian

    def fit(self, Xtrain, ytrain, batch_size=1000):
        num_samples = Xtrain.shape[0]
        self.Xtrain = Xtrain
        self.ytrain = ytrain
        
        indices = np.arange(num_samples)
        np.random.shuffle(indices)
        self.indices = indices
        
        subset_indices = indices[:batch_size]
        Xtrain_subset = Xtrain[subset_indices]
        ytrain_subset = ytrain[subset_indices]
        
        K = self.kernel(Xtrain_subset, Xtrain_subset)
        self.sol = solve(K + self.reg * np.eye(len(K)), ytrain_subset).T
        
        self.Xtrain_subset = Xtrain_subset
        
        return self

    def predict(self, X, batch_size=1000):
        num_samples = X.shape[0]
        K = np.zeros((self.Xtrain_subset.shape[0], num_samples))
        
        for i in range(0, self.Xtrain_subset.shape[0], batch_size):
            for j in tqdm.tqdm(range(0, num_samples, batch_size)):
                K[i:i+batch_size, j:j+batch_size] = self.kernel(self.Xtrain_subset[i:i+batch_size], X[j:j+batch_size])
        
        return (self.sol @ K).T


train_input1 = np.vstack(train_inputs['combined']) #metadata_df.iloc[0]['level_9']
train_out1 = np.vstack(train_outputs['level_9']) #metadata_df.iloc[2]['level_9']


print(train_input1.shape)
print(train_out1.shape)

model1 = NTK(reg=0.01, alpha=1.0)
# Fit model and predict using batched training
model1 = model1.fit(train_input1, train_out1, batch_size=10000)


(6416962, 34)
(6416962, 20)


In [ ]:
from sklearn.metrics import r2_score

train_predict1 = model1.predict(train_input1)
test_predict1 = model1.predict(test_input1)
train_r21 = r2_score(train_out1, train_predict1)
test_r21 = r2_score(test_out1, test_predict1)
print(f"Train_2 R2:  {train_r21:.4f}, Test_2 R2:  {test_r21:.4f}")


 67%|██████▋   | 4325/6417 [08:08<03:50,  9.07it/s]

In [192]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from numpy.linalg import solve
import tqdm

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class NTK:
    def __init__(self, reg=1, sigma=1.0, alpha=0.5):
        self.reg = reg
        self.sigma = sigma
        self.alpha = alpha
        self.sol = None
        self.Xtrain = None
        self.ytrain = None

    def ntk_kernel(self, pair1, pair2):
        out = pair1 @ pair2.T + 1
        N1 = np.sum(np.power(pair1, 2), axis=-1).reshape(-1, 1) + 1
        N2 = np.sum(np.power(pair2, 2), axis=-1).reshape(-1, 1) + 1

        XX = np.sqrt(N1 @ N2.T)
        out = out / XX

        out = np.clip(out, -1, 1)

        first = (
            1
            / np.pi
            * (out * (np.pi - np.arccos(out)) + np.sqrt(1.0 - np.power(out, 2)))
            * XX
        )
        sec = 1 / np.pi * out * (np.pi - np.arccos(out)) * XX
        out = first + sec

        C = 1
        return out / C

    def gaussian_kernel(self, pair1, pair2):
        sq_dist = np.sum(pair1**2, axis=1).reshape(-1, 1) + np.sum(pair2**2, axis=1) - 2 * np.dot(pair1, pair2.T)
        return np.exp(-sq_dist / (2 * self.sigma**2))

    def kernel(self, pair1, pair2):
        ntk = self.ntk_kernel(pair1, pair2)
        gaussian = self.gaussian_kernel(pair1, pair2)
        return self.alpha * ntk + (1 - self.alpha) * gaussian

    def fit(self, Xtrain, ytrain, batch_size=1000, epochs=10):
        import tqdm
        num_samples = Xtrain.shape[0]
        self.Xtrain = Xtrain
        self.ytrain = ytrain
        
        for epoch in range(epochs):
            indices = np.arange(num_samples)
            np.random.shuffle(indices)
            
            for i in tqdm.tqdm(range(0, num_samples, batch_size)):
                subset_indices = indices[i:i+batch_size]
                Xtrain_subset = Xtrain[subset_indices]
                ytrain_subset = ytrain[subset_indices]
                
                K = self.kernel(Xtrain_subset, Xtrain_subset)
                sol = solve(K + self.reg * np.eye(len(K)), ytrain_subset).T
                
                if self.sol is None:
                    self.sol = sol
                    self.Xtrain_subset = Xtrain_subset
                else:
                    self.sol = np.vstack([self.sol, sol])
                    self.Xtrain_subset = np.vstack([self.Xtrain_subset, Xtrain_subset])
                    
            print(f"Epoch [{epoch+1}/{epochs}] completed")

        return self

    def predict(self, X, batch_size=1000):
        num_samples = X.shape[0]
        num_train_samples = self.Xtrain_subset.shape[0]
        K = np.zeros((num_train_samples, num_samples))
        
        for i in tqdm.tqdm(range(0, num_train_samples, batch_size)):
            for j in range(0, num_samples, batch_size):
                K[i:i+batch_size, j:j+batch_size] = self.kernel(self.Xtrain_subset[i:i+batch_size], X[j:j+batch_size])
        
        return (self.sol @ K).T

# # Example data preparation
# train_inputs = pd.DataFrame({
#     'combined': [np.random.randn(3722, 178) for _ in range(3)]
# })
# train_outputs = pd.DataFrame({
#     'level_9': [np.random.randn(3722, 20) for _ in range(3)]
# })

# train_input1 = np.vstack(train_inputs['combined']) #metadata_df.iloc[0]['level_9']
# train_out1 = np.vstack(train_outputs['level_9']) #metadata_df.iloc[2]['level_9']

print(train_input1.shape)
print(train_out1.shape)

model1 = NTK(reg=0.01, alpha=1.0)
# Fit model and predict using batched training
model1 = model1.fit(train_input1, train_out1, batch_size=1000, epochs=1)

from sklearn.metrics import r2_score

train_predict1 = model1.predict(train_input1)
test_predict1 = model1.predict(test_input1)
train_r21 = r2_score(train_out1, train_predict1)
test_r21 = r2_score(test_out1, test_predict1)
print(f"Train R2: {train_r21:.4f}")
print(f"Test R2: {test_r21:.4f}")


(6416962, 178)
(6416962, 20)


 78%|███████▊  | 4982/6417 [1:04:30<18:34,  1.29it/s]


KeyboardInterrupt: 

In [197]:
def predict(self, X, batch_size=1000):
    num_samples = X.shape[0]
    num_train_samples = self.Xtrain_subset.shape[0]
    K = np.zeros((num_train_samples, num_samples))
    
    for i in tqdm.tqdm(range(0, num_train_samples, batch_size)):
        for j in range(0, num_samples, batch_size):
            K[i:i+batch_size, j:j+batch_size] = self.kernel(self.Xtrain_subset[i:i+batch_size], X[j:j+batch_size])
    
    return (self.sol @ K).T

model1.predict = predict

In [198]:
from sklearn.metrics import r2_score

train_predict1 = model1.predict(train_input1)
test_predict1 = model1.predict(test_input1)
train_r21 = r2_score(train_out1, train_predict1)
test_r21 = r2_score(test_out1, test_predict1)
print(f"Train R2: {train_r21:.4f}")
print(f"Test R2: {test_r21:.4f}")


TypeError: predict() missing 1 required positional argument: 'X'

In [193]:
class model2(model1):
    @staticmethod
    def predict(self, X, batch_size=1000):
        num_samples = X.shape[0]
        num_train_samples = self.Xtrain_subset.shape[0]
        K = np.zeros((num_train_samples, num_samples))
        
        for i in tqdm.tqdm(range(0, num_train_samples, batch_size)):
            for j in range(0, num_samples, batch_size):
                K[i:i+batch_size, j:j+batch_size] = self.kernel(self.Xtrain_subset[i:i+batch_size], X[j:j+batch_size])
        
        return (self.sol @ K).T



In [194]:
# train_predict1 = model2.predict(train_input1)
test_predict1 = model2.predict(test_input1)
# train_r21 = r2_score(train_out1, train_predict1)
test_r21 = r2_score(test_out1, test_predict1)
# print(f"Train R2: {train_r21:.4f}")
print(f"Test R2: {test_r21:.4f}")


AttributeError: 'NTK' object has no attribute 'Xtrain_subset'

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from numpy.linalg import solve
import tqdm

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class NTK:
    def __init__(self, reg=1, sigma=1.0, alpha=0.5):
        self.reg = reg
        self.sigma = sigma
        self.alpha = alpha
        self.sol = None
        self.Xtrain_subset = None
        self.ytrain_subset = None

    def ntk_kernel(self, pair1, pair2):
        out = pair1 @ pair2.T + 1
        N1 = np.sum(np.power(pair1, 2), axis=-1).reshape(-1, 1) + 1
        N2 = np.sum(np.power(pair2, 2), axis=-1).reshape(-1, 1) + 1

        XX = np.sqrt(N1 @ N2.T)
        out = out / XX

        out = np.clip(out, -1, 1)

        first = (
            1
            / np.pi
            * (out * (np.pi - np.arccos(out)) + np.sqrt(1.0 - np.power(out, 2)))
            * XX
        )
        sec = 1 / np.pi * out * (np.pi - np.arccos(out)) * XX
        out = first + sec

        C = 1
        return out / C

    def gaussian_kernel(self, pair1, pair2):
        sq_dist = np.sum(pair1**2, axis=1).reshape(-1, 1) + np.sum(pair2**2, axis=1) - 2 * np.dot(pair1, pair2.T)
        return np.exp(-sq_dist / (2 * self.sigma**2))

    def kernel(self, pair1, pair2):
        ntk = self.ntk_kernel(pair1, pair2)
        gaussian = self.gaussian_kernel(pair1, pair2)
        return self.alpha * ntk + (1 - self.alpha) * gaussian

    def fit(self, Xtrain, ytrain, batch_size=1000, epochs=10):
        num_samples = Xtrain.shape[0]
        self.Xtrain = Xtrain
        self.ytrain = ytrain
        
        for epoch in range(epochs):
            indices = np.arange(num_samples)
            np.random.shuffle(indices)
            
            for i in tqdm.tqdm(range(0, num_samples, batch_size)):
                subset_indices = indices[i:i+batch_size]
                Xtrain_subset = Xtrain[subset_indices]
                ytrain_subset = ytrain[subset_indices]
                
                K = self.kernel(Xtrain_subset, Xtrain_subset)
                sol = solve(K + self.reg * np.eye(len(K)), ytrain_subset).T
                
                if self.sol is None:
                    self.sol = sol
                    self.Xtrain_subset = Xtrain_subset
                else:
                    self.sol = np.vstack([self.sol, sol])
                    self.Xtrain_subset = np.vstack([self.Xtrain_subset, Xtrain_subset])
                    
            print(f"Epoch [{epoch+1}/{epochs}] completed")

        return self

    def predict(self, X, batch_size=1000):
        num_samples = X.shape[0]
        num_train_samples = self.Xtrain_subset.shape[0]
        K = np.zeros((num_train_samples, num_samples))
        
        for i in tqdm.tqdm(range(0, num_train_samples, batch_size)):
            for j in range(0, num_samples, batch_size):
                K[i:i+batch_size, j:j+batch_size] = self.kernel(self.Xtrain_subset[i:i+batch_size], X[j:j+batch_size])
        
        return (self.sol @ K).T

print(train_input1.shape)
print(train_out1.shape)

model1 = NTK(reg=0.01, alpha=1.0)
# Fit model and predict using batched training
model1 = model1.fit(train_input1, train_out1, batch_size=1000, epochs=1)

from sklearn.metrics import r2_score

train_predict1 = model1.predict(train_input1)
test_predict1 = model1.predict(test_input1)
train_r21 = r2_score(train_out1, train_predict1)
test_r21 = r2_score(test_out1, test_predict1)
print(f"Train R2: {train_r21:.4f}")
print(f"Test R2: {test_r21:.4f}")


In [127]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from numpy.linalg import solve

# # Device configuration
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# class NTK:
#     def __init__(self, reg=1, sigma=1.0, alpha=0.5):
#         super().__init__()
#         self.reg = reg
#         self.sigma = sigma
#         self.alpha = alpha
#         self.sol = None
#         self.Xtrain = None

#     def ntk_kernel(self, pair1, pair2):
#         out = pair1 @ pair2.T + 1
#         N1 = np.sum(np.power(pair1, 2), axis=-1).reshape(-1, 1) + 1
#         N2 = np.sum(np.power(pair2, 2), axis=-1).reshape(-1, 1) + 1

#         XX = np.sqrt(N1 @ N2.T)
#         out = out / XX

#         out = np.clip(out, -1, 1)

#         first = (
#             1
#             / np.pi
#             * (out * (np.pi - np.arccos(out)) + np.sqrt(1.0 - np.power(out, 2)))
#             * XX
#         )
#         sec = 1 / np.pi * out * (np.pi - np.arccos(out)) * XX
#         out = first + sec

#         C = 1
#         return out / C

#     def gaussian_kernel(self, pair1, pair2):
#         sq_dist = np.sum(pair1**2, axis=1).reshape(-1, 1) + np.sum(pair2**2, axis=1) - 2 * np.dot(pair1, pair2.T)
#         return np.exp(-sq_dist / (2 * self.sigma**2))

#     def kernel(self, pair1, pair2):
#         ntk = self.ntk_kernel(pair1, pair2)
#         gaussian = self.gaussian_kernel(pair1, pair2)
#         return self.alpha * ntk + (1 - self.alpha) * gaussian

#     def fit(self, Xtrain, ytrain):
#         K = self.kernel(Xtrain, Xtrain)
#         sol = solve(K + self.reg * np.eye(len(K)), ytrain).T
#         self.sol = sol
#         self.Xtrain = Xtrain
#         return self

#     def predict(self, X):
#         K = self.kernel(self.Xtrain, X)
#         return (self.sol @ K).T

import torch
import torch.nn as nn
import torch.optim as optim
from numpy.linalg import solve
import numpy as np

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class NTK:
    def __init__(self, reg=1, sigma=1.0, alpha=0.5):
        self.reg = reg
        self.sigma = sigma
        self.alpha = alpha
        self.sol = None
        self.Xtrain = None

    def ntk_kernel(self, pair1, pair2):
        out = pair1 @ pair2.T + 1
        N1 = np.sum(np.power(pair1, 2), axis=-1).reshape(-1, 1) + 1
        N2 = np.sum(np.power(pair2, 2), axis=-1).reshape(-1, 1) + 1

        XX = np.sqrt(N1 @ N2.T)
        out = out / XX

        out = np.clip(out, -1, 1)

        first = (
            1
            / np.pi
            * (out * (np.pi - np.arccos(out)) + np.sqrt(1.0 - np.power(out, 2)))
            * XX
        )
        sec = 1 / np.pi * out * (np.pi - np.arccos(out)) * XX
        out = first + sec

        C = 1
        return out / C

    def gaussian_kernel(self, pair1, pair2):
        sq_dist = np.sum(pair1**2, axis=1).reshape(-1, 1) + np.sum(pair2**2, axis=1) - 2 * np.dot(pair1, pair2.T)
        return np.exp(-sq_dist / (2 * self.sigma**2))

    def kernel(self, pair1, pair2):
        ntk = self.ntk_kernel(pair1, pair2)
        gaussian = self.gaussian_kernel(pair1, pair2)
        return self.alpha * ntk + (1 - self.alpha) * gaussian

    def fit(self, Xtrain, ytrain, batch_size=1000):
        import tqdm
        num_samples = Xtrain.shape[0]
        self.Xtrain = Xtrain
        
        K = np.zeros((num_samples, num_samples))
        
        for i in tqdm.tqdm(range(0, num_samples, batch_size)):
            for j in range(0, num_samples, batch_size):
                K[i:i+batch_size, j:j+batch_size] = self.kernel(Xtrain[i:i+batch_size], Xtrain[j:j+batch_size])
        
        self.sol = solve(K + self.reg * np.eye(len(K)), ytrain).T
        
        return self

    def predict(self, X, batch_size=1000):
        import tqdm
        num_samples = X.shape[0]
        K = np.zeros((self.Xtrain.shape[0], num_samples))
        
        for i in tqdm.tqdm(range(0, self.Xtrain.shape[0], batch_size)):
            for j in range(0, num_samples, batch_size):
                K[i:i+batch_size, j:j+batch_size] = self.kernel(self.Xtrain[i:i+batch_size], X[j:j+batch_size])
        
        return (self.sol @ K).T


In [128]:
train_input1 = np.vstack(train_inputs['combined']) #metadata_df.iloc[0]['level_9']
train_out1 = np.vstack(train_outputs['level_9']) #metadata_df.iloc[2]['level_9']
test_input1 = np.vstack(test_inputs['combined']) #metadata_df.iloc[0]['level_9']
test_out1 = np.vstack(test_outputs['level_9']) #metadata_df.iloc[2]['level_9']

print(train_input1.shape)
print(train_out1.shape)

model1 = NTK(reg=0.01, alpha=1.0)
# Fit model and predict using batched training
model1 = model1.fit(train_input1, train_out1, batch_size=1000)

(6416962, 178)
(6416962, 20)


MemoryError: Unable to allocate 300. TiB for an array with shape (6416962, 6416962) and data type float64

# KAN w/Fourier instead of b-spline (Chat-GPT + https://github.com/GistNoesis/FourierKAN)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset

device = 'cuda' if torch.cuda.is_available() else 'cpu'

class FourierKANLinear(nn.Module):
    def __init__(self, inputdim, outdim, gridsize, addbias=True, smooth_initialization=False):
        super(FourierKANLinear, self).__init__()
        self.gridsize = gridsize
        self.addbias = addbias
        self.inputdim = inputdim
        self.outdim = outdim
        
        grid_norm_factor = (torch.arange(gridsize) + 1)**2 if smooth_initialization else np.sqrt(gridsize)
        
        self.fouriercoeffs = nn.Parameter(torch.randn(2, outdim, inputdim, gridsize) / 
                                          (np.sqrt(inputdim) * grid_norm_factor))
        if self.addbias:
            self.bias = nn.Parameter(torch.zeros(1, outdim))

    def forward(self, x):
        xshp = x.shape
        outshape = xshp[0:-1] + (self.outdim,)
        x = torch.reshape(x, (-1, self.inputdim))

        k = torch.reshape(torch.arange(1, self.gridsize + 1, device=x.device), (1, 1, 1, self.gridsize))
        xrshp = torch.reshape(x, (x.shape[0], 1, x.shape[1], 1))
        
        c = torch.cos(k * xrshp)
        s = torch.sin(k * xrshp)
        
        y = torch.sum(c * self.fouriercoeffs[0:1], (-2, -1))
        y += torch.sum(s * self.fouriercoeffs[1:2], (-2, -1))
        if self.addbias:
            y += self.bias
        
        y = torch.reshape(y, outshape)
        return y

class FourierKAN(nn.Module):
    def __init__(self, layers_hidden, gridsize, addbias=True, smooth_initialization=False):
        super(FourierKAN, self).__init__()
        self.layers = nn.ModuleList()
        for in_features, out_features in zip(layers_hidden, layers_hidden[1:]):
            self.layers.append(FourierKANLinear(in_features, out_features, gridsize, addbias, smooth_initialization))

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

def train_model(model, train_loader, val_loader, learning_rate=0.0001, epochs=100, return_losses=False):
    import tqdm
    model = model.to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    losses = []
    losses_val = []

    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        for inputs, targets in tqdm.tqdm(train_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        if return_losses:
            losses.append(epoch_loss / len(train_loader))

        if epoch % 10 == 0:
            model.eval()
            val_loss = 0
            with torch.no_grad():
                for val_inputs, val_targets in val_loader:
                    val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
                    val_outputs = model(val_inputs)
                    val_loss += criterion(val_outputs, val_targets).item()
            val_loss /= len(val_loader)
            losses_val.append(val_loss)

            if (epoch % 10 == 0) or (epoch == epochs - 1):
                train_r2, _ = evaluate_model(model, train_loader.dataset.tensors[0].cpu().numpy(), train_loader.dataset.tensors[1].cpu().numpy())
                val_r2, _ = evaluate_model(model, val_loader.dataset.tensors[0].cpu().numpy(), val_loader.dataset.tensors[1].cpu().numpy())
                print(f'Epoch [{epoch}/{epochs}], Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Train R2: {train_r2:.4f}, Val R2: {val_r2:.4f}')

    if return_losses:
        return losses, losses_val

from torch.utils.data import DataLoader, TensorDataset

def evaluate_model(model, input_data, target_data=None, batch_size=1000):
    import tqdm
    model.eval()
    dataset = TensorDataset(torch.tensor(input_data).float(), torch.tensor(target_data).float()) if target_data is not None else TensorDataset(torch.tensor(input_data).float())
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    all_outputs = []
    all_targets = []

    with torch.no_grad():
        for batch in tqdm.tqdm(dataloader):
            if target_data is not None:
                inputs, targets = batch
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                all_targets.append(targets.cpu().numpy())
            else:
                inputs = batch[0]
                inputs = inputs.to(device)
                outputs = model(inputs)
            
            all_outputs.append(outputs.cpu().numpy())
    
    all_outputs = np.vstack(all_outputs)
    
    if target_data is not None:
        all_targets = np.vstack(all_targets)
        return r2_score(all_targets, all_outputs), all_outputs
    else:
        return all_outputs

# Prepare data for training
def prepare_dataloader(train_input, train_output, test_input, test_output, batch_size):
    train_dataset = TensorDataset(torch.tensor(train_input).float(), torch.tensor(train_output).float())
    val_dataset = TensorDataset(torch.tensor(test_input).float(), torch.tensor(test_output).float())

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader

train_input1 = np.vstack(train_inputs['combined']) #metadata_df.iloc[0]['level_9']
train_out1 = np.vstack(train_outputs['level_9']) #metadata_df.iloc[2]['level_9']
test_input1 = np.vstack(test_inputs['combined']) #metadata_df.iloc[0]['level_9']
test_out1 = np.vstack(test_outputs['level_9']) #metadata_df.iloc[2]['level_9']

print(train_input1.shape)
print(train_out1.shape)

# Prepare DataLoader
batch_size = 165
train_loader, val_loader = prepare_dataloader(train_input1, train_out1, test_input1, test_out1, batch_size)

# Model definition
input_size = train_input1.shape[1]
output_size = train_out1.shape[1]
layers_hidden = [input_size, 10, 10, output_size]
gridsize = 15

model1 = FourierKAN(layers_hidden, gridsize, smooth_initialization=True)

# Training
learning_rate = 0.1
epochs = 100

train_model(model1, train_loader, val_loader, learning_rate=learning_rate, epochs=epochs)

# Evaluation
train_r2, train_predict1 = evaluate_model(model1, train_input1, train_out1)
test_r2, test_predict1 = evaluate_model(model1, test_input1, test_out1)

print(f"Train R2: {train_r2:.4f}, Test R2: {test_r2:.4f}")


(6416962, 34)
(6416962, 20)


100%|██████████| 2541/2541 [00:19<00:00, 130.49it/s]


Epoch [0/100], Loss: 32869.7925, Val Loss: 0.8677, Train R2: -25.6559, Val R2: -24.9560


100%|██████████| 2541/2541 [00:18<00:00, 139.79it/s]


Epoch [10/100], Loss: 32873.8759, Val Loss: 0.8407, Train R2: -27.9149, Val R2: -27.4229


 48%|████▊     | 18589/38891 [01:03<01:12, 281.09it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

 23%|██▎       | 8985/38891 [00:32<01:59, 250.19it/s]

In [273]:
model1 = model1.to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model1.parameters(), lr=learning_rate)
losses = []
losses_val = []

for epoch in range(epochs):
    model1.eval()
    epoch_loss = 0
    for inputs, targets in tqdm.tqdm(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model1(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        break
    break

  0%|          | 0/38891 [00:00<?, ?it/s]


In [274]:
outputs

tensor([[-0.1995, -0.2335,  2.1997,  ...,  0.2881, -0.1221,  0.1318],
        [ 1.0030, -0.3096,  0.2798,  ...,  0.9457,  1.8414,  2.0253],
        [ 0.1408,  0.9058,  1.3564,  ...,  0.2161,  0.3684,  0.4066],
        ...,
        [-0.2018, -0.9146,  1.1333,  ..., -0.0602,  0.1246, -0.0916],
        [ 0.7301, -0.0739, -0.6566,  ..., -0.2857,  0.2411,  0.0655],
        [-0.1189, -0.7444,  1.0237,  ...,  0.1279,  1.0676,  1.7605]],
       device='cuda:0', grad_fn=<ViewBackward0>)

In [275]:
loss

tensor(0.8690, device='cuda:0', grad_fn=<MseLossBackward0>)

# Transformer

In [231]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset

device = 'cuda' if torch.cuda.is_available() else 'cpu'

class TransformerModel(nn.Module):
    def __init__(self, inputdim, outdim, num_layers=4, nhead=4, dim_feedforward=128, dropout=0.1):
        super(TransformerModel, self).__init__()
        self.inputdim = inputdim
        self.outdim = outdim
        self.model_type = 'Transformer'
        
        self.input_projection = nn.Linear(inputdim, dim_feedforward)
        self.input_norm = nn.LayerNorm(dim_feedforward)
        self.positional_encoding = nn.Parameter(self._generate_positional_encoding(1, dim_feedforward), requires_grad=False)

        encoder_layers = nn.TransformerEncoderLayer(d_model=dim_feedforward, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=num_layers)
        self.output_norm = nn.LayerNorm(dim_feedforward)

        self.decoder = nn.Linear(dim_feedforward, outdim)

    def _generate_positional_encoding(self, max_seq_len, dim_feedforward):
        position = torch.arange(0, max_seq_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, dim_feedforward, 2) * -(np.log(10000.0) / dim_feedforward))
        pe = torch.zeros(max_seq_len, dim_feedforward)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        return pe
    
    def forward(self, x):
        x = self.input_projection(x).unsqueeze(1)  # Add a sequence length dimension
        x = self.input_norm(x)
        seq_len = x.size(1)
        x = x + self.positional_encoding[:, :seq_len]
        x = self.transformer_encoder(x)
        x = self.output_norm(x)
        x = self.decoder(x).squeeze(1)  # Remove the sequence length dimension
        return x

def train_model(model, train_loader, val_loader, learning_rate=0.0001, epochs=100, return_losses=False):
    model = model.to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    losses = []
    losses_val = []

    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        for inputs, targets in tqdm.tqdm(train_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        if return_losses:
            losses.append(epoch_loss / len(train_loader))

        if epoch % 10 == 0:
            model.eval()
            val_loss = 0
            with torch.no_grad():
                for val_inputs, val_targets in val_loader:
                    val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
                    val_outputs = model(val_inputs)
                    val_loss += criterion(val_outputs, val_targets).item()
            val_loss /= len(val_loader)
            losses_val.append(val_loss)

            if (epoch % 10 == 0) or (epoch == epochs - 1):
                train_r2, _ = evaluate_model(model, train_loader.dataset.tensors[0].cpu().numpy(), train_loader.dataset.tensors[1].cpu().numpy())
                val_r2, _ = evaluate_model(model, val_loader.dataset.tensors[0].cpu().numpy(), val_loader.dataset.tensors[1].cpu().numpy())
                print(f'Epoch [{epoch}/{epochs}], Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Train R2: {train_r2:.4f}, Val R2: {val_r2:.4f}')

    if return_losses:
        return losses, losses_val

def evaluate_model(model, input_data, target_data=None, batch_size=1000):
    model.eval()
    dataset = TensorDataset(torch.tensor(input_data).float(), torch.tensor(target_data).float()) if target_data is not None else TensorDataset(torch.tensor(input_data).float())
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    all_outputs = []
    all_targets = []

    with torch.no_grad():
        for batch in tqdm.tqdm(dataloader):
            if target_data is not None:
                inputs, targets = batch
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                all_targets.append(targets.cpu().numpy())
            else:
                inputs = batch[0]
                inputs = inputs.to(device)
                outputs = model(inputs)
            
            all_outputs.append(outputs.cpu().numpy())
    
    all_outputs = np.vstack(all_outputs)
    
    if target_data is not None:
        all_targets = np.vstack(all_targets)
        return r2_score(all_targets, all_outputs), all_outputs
    else:
        return all_outputs

def prepare_dataloader(train_input, train_output, test_input, test_output, batch_size):
    train_dataset = TensorDataset(torch.tensor(train_input).float(), torch.tensor(train_output).float())
    val_dataset = TensorDataset(torch.tensor(test_input).float(), torch.tensor(test_output).float())

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader

train_input1 = np.vstack(train_inputs['combined']) #metadata_df.iloc[0]['level_9']
train_out1 = np.vstack(train_outputs['level_9']) #metadata_df.iloc[2]['level_9']
test_input1 = np.vstack(test_inputs['combined']) #metadata_df.iloc[0]['level_9']
test_out1 = np.vstack(test_outputs['level_9']) #metadata_df.iloc[2]['level_9']

print(train_input1.shape)

# Model definition
input_size = train_input1.shape[1]
output_size = train_out1.shape[1]
num_layers = 50
nhead = 8
dim_feedforward = 256
dropout = 0.1

model1 = TransformerModel(input_size, output_size, num_layers, nhead, dim_feedforward, dropout)

# Training
learning_rate = 0.0001
epochs = 100

train_model(model1, train_loader, val_loader, learning_rate=learning_rate, epochs=epochs)

# Evaluation
train_r2, train_predict1 = evaluate_model(model1, train_input1, train_out1)
test_r2, test_predict1 = evaluate_model(model1, test_input1, test_out1)

print(f"Train R2: {train_r2:.4f}, Test R2: {test_r2:.4f}")


/msc/home/vsharm64/miniconda3/envs/AiTLS/lib/python3.10/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
100%|██████████| 12/12 [00:00<00:00, 40.35it/s]


Epoch [0/100], Loss: 70.4119, Val Loss: 1.0009, Train R2: -0.0015, Val R2: -0.0016


100%|██████████| 12/12 [00:00<00:00, 39.15it/s]


Epoch [10/100], Loss: 68.6323, Val Loss: 1.0012, Train R2: -0.0018, Val R2: -0.0020


100%|██████████| 12/12 [00:00<00:00, 32.93it/s]


...

Epoch [80/100], Loss: 68.1368, Val Loss: 1.0011, Train R2: -0.0014, Val R2: -0.0019


100%|██████████| 12/12 [00:00<00:00, 38.56it/s]


Epoch [90/100], Loss: 68.0977, Val Loss: 1.0001, Train R2: -0.0008, Val R2: -0.0009


100%|██████████| 12/12 [00:00<00:00, 40.74it/s]

Train R2: -0.0011, Test R2: -0.0012


In [244]:
train_out1.shape

(11166, 20)

In [235]:
input_size = train_input1.shape[1]
output_size = train_out1.shape[1]
num_layers = 50
nhead = 8
dim_feedforward = 256
dropout = 0.1

model = TransformerModel(input_size, output_size, num_layers, nhead, dim_feedforward, dropout)

model = model.to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
losses = []
losses_val = []

for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for inputs, targets in tqdm.tqdm(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    if return_losses:
        losses.append(epoch_loss / len(train_loader))

    if epoch % 10 == 0:
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for val_inputs, val_targets in val_loader:
                val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
                val_outputs = model(val_inputs)
                val_loss += criterion(val_outputs, val_targets).item()
        val_loss /= len(val_loader)
        losses_val.append(val_loss)

        if (epoch % 10 == 0) or (epoch == epochs - 1):
            train_r2, _ = evaluate_model(model, train_loader.dataset.tensors[0].cpu().numpy(), train_loader.dataset.tensors[1].cpu().numpy())
            val_r2, _ = evaluate_model(model, val_loader.dataset.tensors[0].cpu().numpy(), val_loader.dataset.tensors[1].cpu().numpy())
            print(f'Epoch [{epoch}/{epochs}], Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Train R2: {train_r2:.4f}, Val R2: {val_r2:.4f}')

if return_losses:
    return losses, losses_val

  0%|          | 0/68 [00:00<?, ?it/s]


In [238]:
inputs.shape

torch.Size([165, 178])

In [239]:
targets.shape

torch.Size([165, 20])

In [242]:
outputs.shape

torch.Size([165, 20])

# Conditional Transformer

In [219]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import r2_score
import tqdm
from torch.utils.data import DataLoader, TensorDataset

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class ConditionalTransformer(nn.Module):
    def __init__(self, input_dim, extra_dim, output_dim, d_model=256, nhead=8, num_encoder_layers=4, num_decoder_layers=4, dim_feedforward=512, dropout=0.1):
        super(ConditionalTransformer, self).__init__()
        self.input_dim = input_dim
        self.extra_dim = extra_dim
        self.d_model = d_model
        self.output_dim = output_dim

        self.encoder_embedding = nn.Linear(input_dim + extra_dim, d_model)
        self.decoder_embedding = nn.Linear(output_dim, d_model)
        self.positional_encoding = nn.Parameter(torch.zeros(1, 5000, d_model))

        self.transformer = nn.Transformer(d_model=d_model, nhead=nhead, num_encoder_layers=num_encoder_layers, num_decoder_layers=num_decoder_layers, dim_feedforward=dim_feedforward, dropout=dropout)
        
        self.fc_out = nn.Linear(d_model, output_dim)

    def forward(self, src, extra, tgt):
        src_seq_len, tgt_seq_len = src.shape[1], tgt.shape[1]

        # Concatenate extra features with the input
        extra_expanded = extra.unsqueeze(1).expand(-1, src_seq_len, -1)
        src = torch.cat((src, extra_expanded), dim=-1)

        src_emb = self.encoder_embedding(src) + self.positional_encoding[:, :src_seq_len, :]
        tgt_emb = self.decoder_embedding(tgt) + self.positional_encoding[:, :tgt_seq_len, :]

        src_emb = src_emb.permute(1, 0, 2)
        tgt_emb = tgt_emb.permute(1, 0, 2)

        output = self.transformer(src_emb, tgt_emb)
        output = self.fc_out(output.permute(1, 0, 2))
        
        return output

def train_model(model, train_loader, val_loader, learning_rate=0.0001, epochs=100, return_losses=False):
    model = model.to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    losses = []
    losses_val = []

    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        for src, extra, tgt in tqdm.tqdm(train_loader):
            src, extra, tgt = src.to(device), extra.to(device), tgt.to(device)
            optimizer.zero_grad()
            outputs = model(src, extra, tgt)
            loss = criterion(outputs, tgt)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        if return_losses:
            losses.append(epoch_loss / len(train_loader))

        if epoch % 10 == 0:
            model.eval()
            val_loss = 0
            with torch.no_grad():
                for val_src, val_extra, val_tgt in val_loader:
                    val_src, val_extra, val_tgt = val_src.to(device), val_extra.to(device), val_tgt.to(device)
                    val_outputs = model(val_src, val_extra, val_tgt)
                    val_loss += criterion(val_outputs, val_tgt).item()
            val_loss /= len(val_loader)
            losses_val.append(val_loss)

            if (epoch % 100 == 0) or (epoch == epochs - 1):
                train_r2, _ = evaluate_model(model, train_loader.dataset.tensors[0].cpu().numpy(), train_loader.dataset.tensors[1].cpu().numpy(), train_loader.dataset.tensors[2].cpu().numpy())
                val_r2, _ = evaluate_model(model, val_loader.dataset.tensors[0].cpu().numpy(), val_loader.dataset.tensors[1].cpu().numpy(), val_loader.dataset.tensors[2].cpu().numpy())
                print(f'Epoch [{epoch}/{epochs}], Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Train R2: {train_r2:.4f}, Val R2: {val_r2:.4f}')

    if return_losses:
        return losses, losses_val

def evaluate_model(model, input_data, extra_data, target_data=None, batch_size=1000):
    model.eval()
    dataset = TensorDataset(torch.tensor(input_data).float(), torch.tensor(extra_data).float(), torch.tensor(target_data).float()) if target_data is not None else TensorDataset(torch.tensor(input_data).float(), torch.tensor(extra_data).float())
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    all_outputs = []
    all_targets = []

    with torch.no_grad():
        for batch in tqdm.tqdm(dataloader):
            if target_data is not None:
                src, extra, targets = batch
                src, extra, targets = src.to(device), extra.to(device), targets.to(device)
                outputs = model(src, extra, targets)
                all_targets.append(targets.cpu().numpy())
            else:
                src, extra = batch
                src, extra = src.to(device), extra.to(device)
                outputs = model(src, extra, torch.zeros_like(src))
            
            all_outputs.append(outputs.cpu().numpy())
    
    all_outputs = np.vstack(all_outputs)
    
    if target_data is not None:
        all_targets = np.vstack(all_targets)
        return r2_score(all_targets, all_outputs), all_outputs
    else:
        return all_outputs

def prepare_dataloader(original_inputs, conditionals, outputs, batch_size, shuffle=True):
    dataset = TensorDataset(torch.tensor(original_inputs).float(), torch.tensor(conditionals).float(), torch.tensor(outputs).float())
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return loader

# Extract the original input (first 8 dimensions) and the 170 one-hot encoded variables
def extract_inputs_and_conditionals(data):
    original_inputs = data[:, :8]
    conditionals = data[:, 8:]
    return original_inputs, conditionals

# Example data preparation
train_inputs = pd.DataFrame({
    'combined': [np.random.randn(3722, 178) for _ in range(3)]
})
train_outputs = pd.DataFrame({
    'level_9': [np.random.randn(3722, 20) for _ in range(3)]
})
test_inputs = pd.DataFrame({
    'combined': [np.random.randn(3722, 178) for _ in range(3)]
})
test_outputs = pd.DataFrame({
    'level_9': [np.random.randn(3722, 20) for _ in range(3)]
})

train_input1 = np.vstack(train_inputs['combined']) #metadata_df.iloc[0]['level_9']
train_out1 = np.vstack(train_outputs['level_9']) #metadata_df.iloc[2]['level_9']
test_input1 = np.vstack(test_inputs['combined']) #metadata_df.iloc[0]['level_9']
test_out1 = np.vstack(test_outputs['level_9']) #metadata_df.iloc[2]['level_9']

# Extract the original inputs and conditionals
train_original_inputs, train_conditionals = extract_inputs_and_conditionals(train_input1)
test_original_inputs, test_conditionals = extract_inputs_and_conditionals(test_input1)

# Print shapes to verify
print(f"Train Original Inputs Shape: {train_original_inputs.shape}")
print(f"Train Conditionals Shape: {train_conditionals.shape}")
print(f"Train Outputs Shape: {train_out1.shape}")
print(f"Test Original Inputs Shape: {test_original_inputs.shape}")
print(f"Test Conditionals Shape: {test_conditionals.shape}")
print(f"Test Outputs Shape: {test_out1.shape}")

# Prepare DataLoader
batch_size = 64
train_loader = prepare_dataloader(train_original_inputs, train_conditionals, train_out1, batch_size)
val_loader = prepare_dataloader(test_original_inputs, test_conditionals, test_out1, batch_size, shuffle=False)

# Model definition
input_size = train_original_inputs.shape[1]
extra_size = train_conditionals.shape[1]
output_size = train_out1.shape[1]

model_T = ConditionalTransformer(input_dim=input_size, extra_dim=extra_size, output_dim=output_size)

# Training
learning_rate = 0.01
epochs = 2

train_model(model_T, train_loader, val_loader, learning_rate=learning_rate, epochs=epochs)

# Evaluation
train_r2, train_predict1 = evaluate_model(model_T, train_original_inputs, train_conditionals, train_out1)
test_r2, test_predict1 = evaluate_model(model_T, test_original_inputs, test_conditionals, test_out1)

print(f"Train R2: {train_r2:.4f}, Test R2: {test_r2:.4f}")


/msc/home/vsharm64/miniconda3/envs/AiTLS/lib/python3.10/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Train Original Inputs Shape: (11166, 8)
Train Conditionals Shape: (11166, 170)
Train Outputs Shape: (11166, 20)
Test Original Inputs Shape: (11166, 8)
Test Conditionals Shape: (11166, 170)
Test Outputs Shape: (11166, 20)


  0%|          | 0/175 [00:00<?, ?it/s]


RuntimeError: Tensors must have same number of dimensions: got 2 and 3